In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [2]:
# Scrape wiki page

website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = bs(website_url, 'lxml')

In [3]:
# Isolate table of interest

canada_neigh = soup.find('table',{'class':'wikitable sortable'})
rows = canada_neigh.find_all('tr')

In [4]:
# Make empty pandas dataframe
can_neigh = pd.DataFrame(columns=['Postalcode', 'Borough', 'Neighborhood'], index=range(0,288)) 


# Populate can_neigh dataframe with wiki table
row_marker = 0
for row in rows[1:]:
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
        can_neigh.iat[row_marker,column_marker] = column.get_text().rstrip()
        column_marker +=1
    row_marker +=1

# Drop rows where Borough is "Not assigned"
can_neigh = can_neigh[can_neigh['Borough'] != "Not assigned"]

# Assign value of borough for neighborhoods with value "Not assigned"
i = can_neigh['Neighborhood'].loc[can_neigh.Neighborhood=='Not assigned'].index.values
for x in i:
    can_neigh['Neighborhood'][x] = can_neigh['Borough'][x]
can_neigh

# Get rid of duplicate postalcodes and merge neighborhoods
can_neigh=can_neigh.groupby(by=['Postalcode','Borough'])['Neighborhood'].agg(lambda col: ','.join(col))
can_neigh = pd.DataFrame(can_neigh).reset_index()

In [5]:
# Find shape of can_neigh dataframe 
can_neigh.shape

(103, 3)